# Неделя 10 • Natural Language Processing
## Классификация текстов
### Transformers

## Классификация текста с помощью BERT

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv', index_col='Unnamed: 0')
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
df = df[['class', 'tweet']]
df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import torch
from torch import nn
import matplotlib.pyplot as plt

# импортируем трансформеры
import transformers
import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


### Загружаем модель

Список предобученных моделей: [https://huggingface.co/transformers/pretrained_models.html](https://huggingface.co/transformers/pretrained_models.html)

In [ ]:
# https://huggingface.co/google-bert/bert-base-uncased

# подгружаем токенизатор и модель
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained("bert-base-uncased")

# Возьмем произвольный текст из примера
text = "Replace me by any text you'd like."

# Токенизируем текст
encoded_input = tokenizer(text, return_tensors='pt')

In [ ]:
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
tokenizer.decode(102)

'[ S E P ]'

In [ ]:
encoded_tweets = df['tweet'].apply(
    lambda x: tokenizer(x, max_length=64, truncation=True, padding='max_length')
).values

In [ ]:
print(f"Source text: {df['tweet'][10]}")
print(f'Encoded text: {encoded_tweets[10]}')

Source text: " Keeks is a bitch she curves everyone " lol I walked into a conversation like this. Smh
Encoded text: {'input_ids': [101, 1000, 17710, 5937, 2015, 2003, 1037, 7743, 2016, 10543, 3071, 1000, 8840, 2140, 1045, 2939, 2046, 1037, 4512, 2066, 2023, 1012, 15488, 2232, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
tokenizer.decode(7743)

'b i t c h'

Задаем датасет, чтобы удобнее было отправлять в модель данные батчами

In [ ]:
len(encoded_tweets)

24783

In [ ]:
class BertInputs(torch.utils.data.Dataset):
    def __init__(self, encoded_tweets):
        super().__init__()
        self.inputs = encoded_tweets

    def __len__(self):
        return self.inputs.shape[0]

    def __getitem__(self, idx):
        # print(self.inputs[idx])
        return (torch.Tensor(self.inputs[idx]['input_ids']).cuda().long(),
                torch.Tensor(self.inputs[idx]['attention_mask']).cuda().long())

dataset = BertInputs(encoded_tweets)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False)
batch = next(iter(loader))
batch
# shape BATCH_SIZE x MAX_LEN

[tensor([[ 101,  999,  999,  ...,    0,    0,    0],
         [ 101,  999,  999,  ...,    0,    0,    0],
         [ 101,  999,  999,  ...,    0,    0,    0],
         ...,
         [ 101, 1000, 1030,  ...,    0,    0,    0],
         [ 101, 1000, 1030,  ...,    0,    0,    0],
         [ 101, 1000, 1030,  ..., 1025,  102,    0]], device='cuda:0'),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 0]], device='cuda:0')]

In [ ]:
model.cuda();

Бежим по всему загрузчику и отправляем данные в модель. Векторные представления переводим в `numpy`-массивы и добавляем в список `features` – это будущие данные для классификации.

In [ ]:
%%time
features = []
for inputs, masks in loader:
    with torch.inference_mode():
        model_out = model(inputs, attention_mask=masks)
        vectors = model_out.last_hidden_state[:, 0, :]
    features.extend(vectors.cpu().numpy())
len(features)

CPU times: user 1min 21s, sys: 97.5 ms, total: 1min 22s
Wall time: 1min 22s


24783

In [ ]:
# features[0]

(768,)

### Объяснение индексации

In [ ]:
model_out.last_hidden_state.shape

torch.Size([128, 64, 768])

In [ ]:
last_hidden_states[0][:, 0, :]

tensor([[-0.1158,  0.2209, -0.1921,  ..., -0.1910,  0.3733,  0.3903],
        [ 0.0308,  0.2985, -0.0837,  ..., -0.0993,  0.4893,  0.1393],
        [ 0.1911, -0.0805, -0.0236,  ..., -0.0527,  0.4683,  0.2539],
        ...,
        [-0.0315,  0.2116, -0.0864,  ..., -0.1650,  0.4163,  0.2819],
        [-0.0673,  0.1769, -0.1371,  ..., -0.2006,  0.3644,  0.2703],
        [-0.3547, -0.3258,  0.1044,  ..., -0.0622,  0.2506,  0.3589]],
       device='cuda:0')

Почему именно такая индексация `last_hidden_states`? Читаем в оригинальной статье 📝 [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805):

![](https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png)

❗️❗️❗️ ИМЕННО ТАКАЯ ИНДЕКСАЦИЯ ТОЛЬКО в BERT, для аналогичных моделей надо изучать документацию

In [ ]:
model_out_shape = last_hs[0].shape
print(model_out_shape)
print(f'{model_out_shape[0]} - число объектов,\n \
{model_out_shape[1]} - длина каждой последовательности, \n \
{model_out_shape[2]} - длина выходного вектора BERT для одного элемента последовательности')

torch.Size([32, 64, 768])
32 - число объектов,
 64 - длина каждой последовательности, 
 768 - длина выходного вектора BERT для одного элемента последовательности


### Используем выход BERT для обучения классического классификатора

In [ ]:
# n_texts x 768 -> новые признаки для текста

In [ ]:
len(features)

24783

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(features, df['class'])
print(f'Features shape: {len(X_train)}, Target shape: {len(y_train)}')

Features shape: 18587, Target shape: 18587


In [ ]:
df['class'].value_counts()/len(features)*100

,count
class,
1,77.432111
2,16.797805
0,5.770084


In [ ]:
%%time
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

CPU times: user 8.48 s, sys: 1.05 s, total: 9.53 s
Wall time: 6.9 s


0.8526468689477082

In [ ]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

## Построение собственной модели на основе BERT

Можно использовать BERT так же, как были использованы модели на первой неделе фазы (ResNet, Inception, etc). Сделаем его одинм из слоев, заморозим веса и дополним своими слоями.

Пока пример использования модели для русского языка в «чистом» виде (https://huggingface.co/cointegrated/rubert-tiny2)

In [ ]:
# pip install transformers sentencepiece
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)
# (312,)


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

(312,)


In [ ]:
from torch import nn

In [ ]:
s = "Я люблю собак"

t = tokenizer(s, padding=True, truncation=True, return_tensors='pt')
print(t)

{'input_ids': tensor([[    2,   311, 32988, 29999,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


Создание своего класса на основе предобученной модели.

In [ ]:
class MyTinyBERT(nn.Module):
    def __init__(self):
        super().__init__()
        # забираем bert для русского языка
        self.bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
        # морозим его параметры
        for param in self.bert.parameters():
            param.requires_grad = False
        # добавляем собственный слой классификации
        self.linear = nn.Sequential(
            nn.Linear(312, 256),
            nn.Sigmoid(),
            nn.Dropout(),
            # выход на 3 класса как в задаче выше
            nn.Linear(256, 3)
        )

    def forward(self, x):
        # данные на вход берту
        bert_out = self.bert(**{k: v.to(model.device) for k, v in t.items()})
        # нормализируем – хуже не будет
        normed_bert_out = nn.functional.normalize(bert_out.last_hidden_state[:, 0, :])
        # далее блок классификации
        out = self.linear(normed_bert_out)
        print(out.shape)
        return out

In [ ]:
# прогоняем фразу через модель
mybert = MyTinyBERT()
mybert(t)
# выход – три логита для классов

torch.Size([1, 3])


tensor([[ 0.1404,  0.1851, -0.4640]], grad_fn=<AddmmBackward0>)

В таком случае в класс `Dataset` надо добавить выдачу таргетов для конкретной последовательности. Иначе обучить модель не получится.

## Высокоуровненый подход: использование `transformers` для задачи решения классификации

Для многих базовых задач существуют предобученные экземпляры. SST - датасет, размеченный под аналогичную для задачу классификации эмоциональной окраски текстов на английском языке. Если бы решали такую задачу, то в качестве baseline можно было бы взять такую модель.

In [ ]:
automodel = transformers.AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased-finetuned-sst-2-english'
    )
autotoken = transformers.AutoTokenizer.from_pretrained(
    'distilbert-base-uncased-finetuned-sst-2-english'
    )



config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# Это обычный BERT с дополнительным классификационным слоем
automodel

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
input_text = [
    'Cool movie! It was amazing! Great cast, directing and special effects',
    'Awful cast! there was no reason to watch this and waste \
    time on this nonsense',
    'Not bad'
]

input_tokens = autotoken(
    input_text,   # список входящих текстов
    return_tensors='pt', # возвращать сразу в тензорах
    padding=True,
    max_length=10
)
outputs = automodel(**input_tokens)

print(f'Raw model outputs: {outputs}')
print(f'\nSoftmax(logits):{outputs.logits.softmax(dim=-1)}')
print(f'\nIndex of max el: {outputs.logits.argmax(-1)}')
print(f'\n<<Index: class>> {automodel.config.id2label}')
print(f'\nPostprocessed indicies: {[automodel.config.id2label[i.item()] for i in outputs.logits.argmax(-1)]}')

Raw model outputs: SequenceClassifierOutput(loss=None, logits=tensor([[-4.3091,  4.6525],
        [ 4.7159, -3.8237],
        [-3.7709,  4.0237]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Softmax(logits):tensor([[1.2822e-04, 9.9987e-01],
        [9.9980e-01, 1.9553e-04],
        [4.1182e-04, 9.9959e-01]], grad_fn=<SoftmaxBackward0>)

Index of max el: tensor([1, 0, 1])

<<Index: class>> {0: 'NEGATIVE', 1: 'POSITIVE'}

Postprocessed indicies: ['POSITIVE', 'NEGATIVE', 'POSITIVE']


## Использование API HuggingFace

Некоторые модели можно не скачивать, а просто отправлять запрос к сайту HF и получать ответ.
В карточке модели можно нажать кнопку `Deploy` и выбрать `Inference API` (serverless): вам будет предоставлен код, по которому можно обратиться к модели и получить результат.

Возьмем для примера эту модель: https://huggingface.co/DeepPavlov/rubert-base-cased?inference_api=true

В основе этой модели лежит BERT, об этом написано в [статье](https://arxiv.org/pdf/1905.07213).



In [ ]:
from google.colab import userdata
# Этого кода у вас нет, то есть переменной api_key вам нужно
# присвоить собственный ключ, который можно получить после авторизации в сервисе
api_key = userdata.get('hf')

In [ ]:
# api key можно сделать на сайте hf
import requests

API_URL = "https://api-inference.huggingface.co/models/DeepPavlov/rubert-base-cased"
headers = {"Authorization": f"Bearer {api_key}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output = query({
	"inputs": ["Я пошел гулять", "Собака лежит на полу"],
})

Раз в основе лежит BERT, значит нам нужен первый вектор, отвещающий за агрегированное векторное представление всей последовательности.

In [ ]:
# Посмотрим, какой формы вернулись данные
print(np.array(output[0]).shape)
print(np.array(output[1]).shape)

(1, 5, 768)
(1, 6, 768)


Во втором выходе больше слов, поэтому и больше векторов. Забираем для каждой последовательности только первый вектор – они будут длиной 768, также как и в BERT, использованном выше.

In [ ]:
for i in output:
    print(len(i[0][0]), i[0][0])

768 [-0.012261071242392063, 0.06893865019083023, -0.24549978971481323, 0.07102034986019135, 0.34372130036354065, 0.009971534833312035, -0.03405115753412247, 0.08415599912405014, -0.05057305842638016, 0.023713205009698868, 0.11336503177881241, 0.0072334567084908485, -0.08547380566596985, -0.048055533319711685, -0.37688013911247253, 0.02742093987762928, -0.01961776241660118, 0.08898315578699112, 0.017801715061068535, -0.002789214253425598, 0.13296285271644592, -0.041211649775505066, -0.16246676445007324, -0.025279922410845757, -0.049004826694726944, 0.0036429206375032663, 0.12235988676548004, 0.032885998487472534, 0.14708031713962555, 0.07798518985509872, -0.04639618098735809, 0.028985287994146347, -0.07944408059120178, -0.047516703605651855, 0.01579935848712921, 0.04403737187385559, -2.2123591899871826, -0.03389737755060196, -0.0645909458398819, 0.09358450025320053, -0.22474788129329681, -0.035248275846242905, -0.018891919404268265, -0.04954628273844719, -0.08314839005470276, 1.28592979

Не все модели можно использовать с помощью API, а иногда существует и некоторая задержка при отправке или получении запроса. Но это удобный способ быстро првоерить работоспособность проекта.

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
output.last_hidden_state[:, 0, :]

tensor([[ 1.3863e-01,  1.5827e-01, -2.9666e-01, -1.8494e-01, -4.6429e-01,
         -5.0329e-01, -6.2371e-04,  1.0469e+00,  1.7754e-01, -3.9585e-02,
         -2.5098e-01, -1.4966e-01,  7.2869e-02,  1.9662e-01,  1.2086e-01,
          8.5214e-02, -5.2902e-01,  4.4599e-01,  3.2803e-02,  1.2729e-01,
         -1.8406e-01, -1.9723e-01, -1.4141e-02, -4.3873e-02,  3.0995e-01,
          3.6228e-01,  3.2394e-02,  5.5499e-02, -3.6191e-02, -1.9490e-01,
          1.9163e-01,  4.5594e-01, -4.3608e-01, -3.3814e-01,  3.3220e-01,
          1.3876e-01,  3.7784e-01,  7.7509e-02,  7.1753e-02, -1.7745e-01,
         -8.0696e-01, -2.4013e-01,  4.4476e-01,  5.7503e-01,  4.5728e-01,
         -7.4664e-01, -2.6650e+00, -1.1582e-01, -2.4457e-01, -2.5926e-01,
          8.1344e-02, -1.4255e-01,  5.7927e-01,  4.3881e-01,  3.4623e-01,
          2.2726e-01, -5.3047e-01, -2.9969e-03,  1.3115e-01,  2.7000e-01,
         -4.0535e-01,  2.5096e-01, -3.9678e-01, -3.0917e-02, -1.0593e-01,
          5.9698e-01, -1.3360e-01,  8.

In [ ]:
output.pooler_output

tensor([[-0.9377, -0.5043, -0.9799,  0.9030,  0.9329, -0.2438,  0.8926,  0.2288,
         -0.9531, -1.0000, -0.8862,  0.9906,  0.9855,  0.7155,  0.9455, -0.8645,
         -0.6035, -0.6666,  0.3020, -0.1587,  0.7455,  1.0000, -0.4022,  0.4261,
          0.6151,  0.9996, -0.8773,  0.9594,  0.9585,  0.6950, -0.6718,  0.3325,
         -0.9954, -0.2268, -0.9658, -0.9951,  0.6127, -0.7670,  0.0873,  0.0824,
         -0.9518,  0.4713,  1.0000,  0.3299,  0.7583, -0.2670, -1.0000,  0.3166,
         -0.9364,  0.9910,  0.9719,  0.9893,  0.2190,  0.6048,  0.5849, -0.4123,
         -0.0063,  0.1719, -0.3988, -0.6190, -0.6603,  0.5069, -0.9757, -0.9039,
          0.9926,  0.9323, -0.3687, -0.4869, -0.3143,  0.0499,  0.9129,  0.3396,
         -0.1879, -0.9235,  0.8675,  0.3228, -0.6406,  1.0000, -0.7989, -0.9931,
          0.9629,  0.9124,  0.4827, -0.7276,  0.5996, -1.0000,  0.7548, -0.1600,
         -0.9941,  0.3386,  0.8394, -0.4158,  0.2943,  0.6111, -0.5745, -0.7185,
         -0.4768, -0.9681, -